## Install Packages

In [23]:
!pip install -qU crewai crewai-tools langchain-openai langchain-community beautifulsoup4 faiss-cpu selenium undetected-chromedriver

### Selenium

In [24]:
!sudo apt-get update -y
!sudo apt-get install -y chromium-chromedriver
!sudo cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,197 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,269 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,066 kB]


## Config. Environment

In [25]:
import os
from google.colab import userdata

try:
    os.environ["AZURE_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')
    os.environ["AZURE_API_BASE"] = userdata.get('AZURE_OPENAI_ENDPOINT')
    os.environ["AZURE_API_VERSION"] = userdata.get('OPENAI_API_VERSION')
    os.environ["AZURE_DEPLOYMENT_ID"] = userdata.get('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME')
    os.environ["AZURE_EMBEDDING_DEPLOYMENT_NAME"] = userdata.get('AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME') # Add this line
    os.environ["OPENAI_API_TYPE"] = 'azure' # Keep this to explicitly set the provider type for LiteLLM
    EMAIL_ADDRESS = userdata.get('EMAIL_ADDRESS')
    EMAIL_PASSWORD = userdata.get('EMAIL_PASSWORD')

    if not all([os.environ.get("AZURE_OPENAI_API_KEY"), os.environ.get("AZURE_OPENAI_ENDPOINT"), os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"), EMAIL_ADDRESS, EMAIL_PASSWORD]):
        raise ValueError("One or more secrets are missing.")

    print("All secrets loaded successfully!")

except Exception as e:
    print(f"Error loading secrets: {e}. Please check the 'Secrets'.")

Error loading secrets: One or more secrets are missing.. Please check the 'Secrets'.


In [26]:
%%writefile recipients.csv
name,email
Aditya Bayhaqie,adityabayhaqie@gmail.com
Umar Bayhaqie,thisismebayhaqie@gmail.com
Jack Waltz,jackwaltz001@gmail.com

Overwriting recipients.csv


## Web Scraping

In [27]:
import requests
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, NoSuchElementException
from urllib.parse import urljoin, urldefrag
from dateutil.parser import parse as parse_date

In [28]:
# Document class to hold content and metadata
class Document:
    def __init__(self, page_content, metadata):
        self.page_content = page_content
        self.metadata = metadata
    def __repr__(self):
        return f"Document(metadata={self.metadata})"

In [29]:
def scrape_github_releases(api_url):
    documents = []
    try:
        response = requests.get(f"{api_url}?per_page=15", timeout=15)
        response.raise_for_status()
        releases = response.json()
        for release in releases:
            content = f"## {release.get('name', 'Untitled Release')}\\n\\n{release.get('body', 'No description.')}"
            release_date = release.get('published_at', '')
            doc = Document(
                page_content=content,
                metadata={
                    "source": "https://github.com/langflow-ai/langflow/releases",
                    "release_date": release_date.split('T')[0] if release_date else 'unknown'
                }
            )
            documents.append(doc)
        return documents
    except requests.RequestException as e:
        print(f"Error fetching GitHub releases from {api_url}: {e}")
        return []

In [30]:
def scrape_simplidots_with_selenium(base_url, max_depth=2):
    """A more robust scraper for SimpliDOTS with better error handling."""
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)

    all_documents = []
    visited_urls = set()

    def crawl_and_scrape(url, depth):
        if depth > max_depth or url in visited_urls:
            return

        print(f"-> Visiting (Depth {depth}): {url}")
        visited_urls.add(url)

        try:
            driver.get(url)
            # Wait for the main content area to be present before continuing
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.TAG_NAME, "main"))
            )
            time.sleep(2) # Extra wait for any JS rendering

            # Content Extraction
            # Exclude index pages which don't have unique content
            title = driver.title
            if "Fitur pada" in title or "Feature Updates" in title:
                content_text = driver.find_element(By.TAG_NAME, "main").text.strip()
                if len(content_text) > 200: # Ensure it's a content page
                    doc = Document(page_content=content_text, metadata={"source": url})
                    all_documents.append(doc)
                    print(f"Scraped content from: {title.split('|')[0].strip()}")

            # Link Discovery
            if depth < max_depth:
                links = driver.find_elements(By.TAG_NAME, "a")
                urls_to_visit = []
                for link in links:
                    href = link.get_attribute("href")
                    if href and href.startswith(base_url) and href not in visited_urls:
                        urls_to_visit.append(href)

                for next_url in set(urls_to_visit): # Use set to avoid duplicates on the same page
                    crawl_and_scrape(next_url, depth + 1)

        except (TimeoutException, NoSuchElementException) as e:
            print(f"Warning: Could not process page {url}. It might be an index or non-article page. Skipping.")
        except Exception as e:
            print(f"Error processing {url}: {e}")

    try:
        crawl_and_scrape(base_url, 1)
    finally:
        driver.quit()

    return all_documents

In [31]:
def scrape_anthropic_with_selenium(url):
    """A more robust scraper for Anthropic with better waits."""
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)

    try:
        driver.get(url)
        # Wait for a specific, meaningful element like the main content div
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, "//div[h1[text()='API release notes']]"))
        )
        time.sleep(2) # Extra wait

        content_element = driver.find_element(By.TAG_NAME, "body")
        print("Scraped content from: Anthropic")
        return [Document(page_content=content_element.text, metadata={"source": url})]
    except TimeoutException:
        print(f"Error: Timed out waiting for content to load on {url}. The page structure may have changed.")
        return []
    except Exception as e:
        print(f"An error occurred while scraping Anthropic: {e}")
        return []
    finally:
        driver.quit()

In [32]:
URLS = {
    "simplidots": "https://fitur-sap.simplidots.id/",
    "langflow": "https://api.github.com/repos/langflow-ai/langflow/releases",
    "anthropic": "https://docs.anthropic.com/en/release-notes/api"
}
print("Starting data scraping...")
all_documents = []
all_documents.extend(scrape_simplidots_with_selenium(URLS["simplidots"]))
all_documents.extend(scrape_github_releases(URLS["langflow"]))
all_documents.extend(scrape_anthropic_with_selenium(URLS["anthropic"]))
print(f"\\nScraping complete. Total documents found: {len(all_documents)}")

Starting data scraping...
-> Visiting (Depth 1): https://fitur-sap.simplidots.id/
-> Visiting (Depth 2): https://fitur-sap.simplidots.id/smh/fitur-pada-smh-sales-management-hub/2024/penambahan-report-sales-invoice-laporan-faktur-12-januari-2024
Scraped content from: Penambahan Report Sales Invoice (Laporan Faktur) - [12 Januari 2024]
-> Visiting (Depth 2): https://fitur-sap.simplidots.id/smh/fitur-pada-smh-sales-management-hub/2023/penambahan-fitur-ekspor-sales-order-ke-.txt-file-extension-pada-smh-03-aug-2023
Scraped content from: Penambahan Fitur Ekspor Sales Order ke .TXT File Extension pada SMH - [03 Aug 2023]
-> Visiting (Depth 2): https://fitur-sap.simplidots.id/smh/fitur-pada-smh-sales-management-hub/2025/penambahan-fitur-log-activity-pengaturan-quantity-jam-mulai-dan-akhir-promo-30-april-2025
Scraped content from: Penambahan Fitur Log Activity, Pengaturan Quantity, Jam Mulai dan Akhir Promo - [30 April 2025]
-> Visiting (Depth 2): https://fitur-sap.simplidots.id/smh/fitur-pada-

## Advanced Preprocessing

In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def clean_text(text):
    text = re.sub(r'\\n\\s*\\n', '\\n\\n', text)
    artifacts = ["Was this helpful?", "Powered by GitBook", "Copy", "Next", "Previous", "Last updated"]
    for artifact in artifacts:
        text = text.replace(artifact, "")
    return text.strip()

In [34]:
def extract_and_format_date(text):
    month_map = {'januari': 'january', 'februari': 'february', 'maret': 'march', 'april': 'april', 'mei': 'may', 'juni': 'june', 'juli': 'july', 'agustus': 'august', 'september': 'september', 'oktober': 'october', 'november': 'november', 'desember': 'december'}
    date_pattern = r"(?i)(\\d{1,2}\\s+(?:Jan(?:uari)?|Feb(?:ruari)?|Mar(?:et)?|Apr(?:il)?|Mei|Jun(?:i)?|Jul(?:i)?|Agu(?:stus)?|Sep(?:tember)?|Okt(?:ober)?|Nov(?:ember)?|Des(?:ember)?)\\s+\\d{4}|(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)\\s+\\d{1,2}(?:st|nd|rd|th)?(?:,)?\\s+\\d{4})"
    match = re.search(date_pattern, text)
    if match:
        try:
            date_str = match.group(0).lower()
            for indo, eng in month_map.items():
                date_str = date_str.replace(indo, eng)
            return parse_date(date_str)
        except (ValueError, TypeError): return None
    return None

In [35]:
# Execute Preprocessing
print(" Starting advanced data preprocessing...")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)
final_chunks = text_splitter.split_documents(all_documents)

for chunk in final_chunks:
    chunk.page_content = clean_text(chunk.page_content)
    if 'release_date' not in chunk.metadata or chunk.metadata['release_date'] == 'unknown':
        extracted_date = extract_and_format_date(chunk.page_content)
        chunk.metadata['release_date'] = extracted_date.strftime('%Y-%m-%d') if extracted_date else 'unknown'

processed_docs = [chunk for chunk in final_chunks if len(chunk.page_content) > 50]
print(f" Preprocessing complete. Total processed chunks: {len(processed_docs)}")

 Starting advanced data preprocessing...
 Preprocessing complete. Total processed chunks: 609


## Filter for Recent Updates

In [36]:
from datetime import datetime

# Updated logic to filter for the current calendar month
today = datetime.now()
first_day_of_month = today.replace(day=1)

monthly_docs = []
for doc in processed_docs:
    release_date_str = doc.metadata.get('release_date')
    if release_date_str and release_date_str != 'unknown':
        try:
            release_date = datetime.strptime(release_date_str, '%Y-%m-%d')
            # Check if the release date is within the current month
            if release_date.year == today.year and release_date.month == today.month:
                monthly_docs.append(doc)
        except ValueError:
            continue

print(f"Found {len(monthly_docs)} documents from the current month (since {first_day_of_month.strftime('%Y-%m-%d')}).")

# Prepare the context for the crew
if monthly_docs:
    # IMPORTANT: Ensure the variable passed to the crew is named 'newsletter_context'
    newsletter_context = "\\n\\n---\\n\\n".join(
        f"Source: {doc.metadata.get('source', 'N/A')}\\n"
        f"Date: {doc.metadata.get('release_date', 'N/A')}\\n\\n"
        f"{doc.page_content}"
        for doc in monthly_docs
    )
else:
    newsletter_context = "No new release notes found in the current month."

Found 42 documents from the current month (since 2025-07-01).


## Configure LLM and Tools

In [37]:
import csv
import smtplib
import time
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from langchain_openai import AzureChatOpenAI
from crewai.tools import BaseTool

try:
    llm = AzureChatOpenAI(
        azure_endpoint=os.environ["AZURE_API_BASE"],
        azure_deployment=os.environ["AZURE_DEPLOYMENT_ID"],
        api_key=os.environ["AZURE_API_KEY"],
        api_version=os.environ["AZURE_API_VERSION"],
        model=f"azure/{userdata.get('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME')}"
    )
    print("Azure LLM initialized successfully.")
except Exception as e:
    print(f"Error initializing Azure LLM: {e}")
    print("Please ensure your Azure OpenAI credentials are set correctly in the cell above.")

Azure LLM initialized successfully.


In [38]:
import csv
import smtplib
import time
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from crewai.tools import BaseTool

# This entire class replaces your old CSVEmailTool
class CSVEmailTool(BaseTool):
    name: str = "Personalized HTML Email Dispatcher"
    description: str = "Reads 'recipients.csv' and sends a personalized email with a custom subject to everyone on the list. The input must be the subject and the newsletter body."

    def _run(self, subject: str, newsletter_body_html: str) -> str:
        # These EMAIL_ADDRESS and EMAIL_PASSWORD variables must be loaded from your secrets
        sender_email = EMAIL_ADDRESS
        sender_password = EMAIL_PASSWORD
        sent_count = 0
        recipient_list = []

        try:
            with open('recipients.csv', mode='r', encoding='utf-8') as csvfile:
                reader = csv.DictReader(csvfile)
                for row in reader:
                    recipient_list.append(row)
            if not recipient_list:
                return "Error: recipients.csv is empty or not found."
        except FileNotFoundError:
            return "Error: recipients.csv not found. Please create it first."
        except Exception as e:
            return f"Error reading CSV file: {e}"

        for recipient in recipient_list:
            recipient_name = recipient.get("name", "there")
            recipient_email = recipient.get("email")

            if not recipient_email:
                continue

            # Create the full HTML document for each recipient
            full_html_content = f"""
            <!DOCTYPE html>
            <html lang="en">
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1.0">
                <style>
                    body {{ font-family: sans-serif; line-height: 1.6; color: #333; }}
                    .container {{ max-width: 600px; margin: 20px auto; padding: 20px; border: 1px solid #ddd; border-radius: 8px; }}
                    h2 {{ color: #0056b3; border-bottom: 1px solid #eee; padding-bottom: 5px;}}
                    h3 {{ color: #444; }}
                    ul {{ padding-left: 20px; }}
                    li {{ margin-bottom: 10px; }}
                    .footer {{ margin-top: 20px; font-size: 0.8em; color: #888; text-align: center; }}
                </style>
            </head>
            <body>
                <div class="container">
                    <p>Hi {recipient_name},</p>
                    <p>Here are the latest updates for this month:</p>
                    {newsletter_body_html}
                    <hr>
                    <p class="footer">To unsubscribe, please reply to this email.</p>
                </div>
            </body>
            </html>
            """

            message = MIMEMultipart()
            message['From'] = f"SimpliDOTS Tech Updates <{sender_email}>"
            message['To'] = recipient_email
            # Use the dynamic subject passed into the tool
            message['Subject'] = subject

            message.attach(MIMEText(full_html_content, 'html'))

            try:
                print(f"Sending '{subject}' to {recipient_name} at {recipient_email}...")
                server = smtplib.SMTP('smtp.gmail.com', 587)
                server.starttls()
                server.login(sender_email, sender_password)
                server.sendmail(sender_email, recipient_email, message.as_string())
                server.quit()
                sent_count += 1
                time.sleep(2) # Wait 2 seconds before sending the next email
            except Exception as e:
                print(f"Failed to send email to {recipient_email}. Error: {e}")

        return f"Successfully sent personalized HTML emails to {sent_count}/{len(recipient_list)} recipients."

# Don't forget to instantiate the tool after the class definition
email_tool = CSVEmailTool()

## Define Agents

In [39]:
## --- ONE EMAIL, THREE RELEASEs ---

# from crewai import Agent, Task

# analyst_agent = Agent(role="Principal Technology Analyst", goal="Analyze provided release notes to identify critical updates.", backstory="You are an expert analyst who extracts impactful information.", llm=llm, verbose=True)

# expert_agent = Agent(role="Expert Tech Newsletter Writer", goal="Craft an engaging newsletter from an analyst's report.", backstory="You are a famous tech writer known for making complex topics exciting.", llm=llm, verbose=True)

# dispatcher_agent = Agent(
#     role="Communications Dispatch Officer",
#     goal="Use the email tool to send the newsletter to all recipients defined in the system's data file.",
#     backstory="You are a reliable specialist ensuring important updates are dispatched correctly.",
#     tools=[email_tool],
#     llm=llm,
#     verbose=True
# )

# analysis_task = Task(
#     description=(
#         "Analyze the provided text which contains software release notes from the current month. "
#         "Your primary job is to group all findings by company (SimpliDots, Langflow, Anthropic). "
#         "Under each company, create sub-categories for 'New Features', 'Bug Fixes', etc., and list the specific updates."
#         "\n\nCONTEXT:\n---\n{context}\n---"
#     ),
#     expected_output=(
#         "A structured, hierarchical report. The top-level categories must be the company names. "
#         "Under each company, there should be sub-categories with bulleted lists of the specific updates."
#     ),
#     agent=analyst_agent
# )

# summarization_task = Task(
#     description=(
#         "Take the analyst's report, which is already categorized by company, and transform it into a polished, professional HTML newsletter body. "
#         "Create a main heading (e.g., `<h2>🚀 Langflow Updates</h2>`) for each company. "
#         "Under each company heading, create subheadings (e.g., `<h3>New Features</h3>`) for the update types. "
#         "Format the details for each update as an unordered list (`<ul>` with `<li>` items). "
#         "Make the title of each list item bold using `<strong>` tags where appropriate. "
#         "Do NOT include the `<html>`, `<head>`, or `<body>` tags, only the content that goes inside the body, starting with the first `<h2>` tag."
#     ),
#     expected_output=(
#         "A string containing the well-formatted HTML for the newsletter body, organized with `<h2>` tags for each company and `<h3>` tags for sub-categories."
#     ),
#     agent=expert_agent,
#     context=[analysis_task]
# )

# email_task = Task(
#     description=(
#         "Take the composed newsletter body and use the Personalized Email Dispatcher tool. "
#         "The tool will automatically find the recipients in the CSV file and send the emails. "
#         "The subject line for the email MUST be 'Monthly Tech Release Notes Digest'."
#     ),
#     expected_output="A confirmation message stating how many emails were successfully sent.",
#     agent=dispatcher_agent,
#     context=[summarization_task]
# )

In [40]:
# --- THREE EMAILs, THREE RELEASES NOTEs

from crewai import Crew, Process, Agent, Task

# Define the products you want to create newsletters for.
# The `source_keyword` is used to filter documents for each product.
products_to_process = [
    {"name": "Langflow", "source_keyword": "github.com/langflow-ai/langflow"},
    {"name": "SimpliDots", "source_keyword": "simplidots.id"},
    {"name": "Anthropic", "source_keyword": "anthropic.com"}
]

# Re-define your agents here to make sure they are in scope for the loop.
# (Ensure the llm variable is already created in a previous cell)
analyst_agent = Agent(role="Principal Technology Analyst", goal="Analyze provided release notes to identify critical updates.", backstory="You are an expert analyst who extracts impactful information.", llm=llm, verbose=True)
expert_agent = Agent(role="Expert Tech Newsletter Writer", goal="Craft an engaging newsletter from an analyst's report.", backstory="You are a famous tech writer known for making complex topics exciting.", llm=llm, verbose=True)
dispatcher_agent = Agent(role="Communications Dispatch Officer", goal="Use the email tool to send the newsletter to all recipients defined in the system's data file.", backstory="You are a reliable specialist ensuring important updates are dispatched correctly.", tools=[email_tool], llm=llm, verbose=True)

## Assemble the Crew

In [41]:
# ONE EMAILs, THREE RELEASES NOTEs

# from crewai import Crew, Process

# release_notes_crew = Crew(
#     agents=[analyst_agent, expert_agent, dispatcher_agent],
#     tasks=[analysis_task, summarization_task, email_task],
#     process=Process.sequential,
#     verbose=True
# )

# # Kick off the crew's work
# print("Kicking off the CSV-Powered Release Notes Crew...")
# if newsletter_context != "No new release notes found in the last 21 days.":
#     try:
#         result = release_notes_crew.kickoff(inputs={'context': newsletter_context})
#         print("\\n\\nCrew execution finished successfully!")
#         print("\\nFinal Result:")
#         print(result)
#     except Exception as e:
#         print(f"\\n\\nAn error occurred during crew execution: {e}")
# else:
#     print("No recent documents found to process. The crew will not run.")

In [42]:
# THREE EMAILs, THREE RELEASES NOTEs

# Loop through each product to create and send a dedicated newsletter
for product in products_to_process:
    product_name = product["name"]
    keyword = product["source_keyword"]

    print(f"\n{'='*60}")
    print(f"Starting process for: {product_name}")
    print(f"{'='*60}")

    # 1. Filter documents for the current product from the current month
    product_docs = [
        doc for doc in monthly_docs if keyword in doc.metadata.get('source', '')
    ]

    if not product_docs:
        print(f"No monthly updates found for {product_name}. Skipping.")
        continue

    print(f"Found {len(product_docs)} document(s) for {product_name}.")

    # 2. Create the context string for this product only
    product_context = "\\n\\n---\\n\\n".join(
        f"Source: {doc.metadata.get('source', 'N/A')}\\n"
        f"Date: {doc.metadata.get('release_date', 'N/A')}\\n\\n"
        f"{doc.page_content}"
        for doc in product_docs
    )

    # 3. Create dynamic tasks specifically for the current product
    analysis_task = Task(
        description=f"Analyze the release notes for '{product_name}' from the current month. "
                    f"Create a clear, bulleted list of all new features, bug fixes, and announcements."
                    f"\n\nCONTEXT:\n---\n{product_context}\n---",
        expected_output=f"A structured report summarizing all updates for {product_name}.",
        agent=analyst_agent
    )

    summarization_task = Task(
        description=f"Take the analyst's report for '{product_name}' and transform it into a polished HTML newsletter body. "
                    f"Use <h3> tags for sub-categories (e.g., 'New Features', 'Bug Fixes'). Use <ul> and <li> for the details. "
                    "Do NOT include <html>, <head>, or <body> tags, only the content that goes inside the body.",
        expected_output=f"A string containing the well-formatted HTML for the {product_name} newsletter body.",
        agent=expert_agent,
        context=[analysis_task]
    )

    email_task = Task(
        description=f"Take the composed newsletter body for '{product_name}' and use the Personalized HTML Email Dispatcher tool. "
                    f"The subject line for the email MUST be 'Monthly {product_name} Release Notes'.",
        expected_output="A confirmation message stating how many emails were successfully sent.",
        agent=dispatcher_agent,
        context=[summarization_task]
    )

    product_crew = Crew(
        agents=[analyst_agent, expert_agent, dispatcher_agent],
        tasks=[analysis_task, summarization_task, email_task],
        process=Process.sequential,
        verbose=1 # Using verbose=1 for cleaner logs in a loop
    )

    print(f"\nKicking off the crew for {product_name}...")
    try:
        result = product_crew.kickoff()
        print(f"\nCrew execution for {product_name} finished successfully!")
        print(f"Final Result: {result}")
    except Exception as e:
        print(f"\nAn error occurred during the {product_name} crew execution: {e}")


Starting process for: Langflow
Found 42 document(s) for Langflow.

Kicking off the crew for Langflow...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 229378c8-caa1-4d63-b748-9f4cc417f3d3                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Principal Technology Analyst                                                                            │
│                                                                                                                 │
│  Task: Analyze the release notes for 'Langflow' from the current month. Create a clear, bulleted list of all    │
│  new features, bug fixes, and announcements.                                                                    │
│                                                                                                                 │
│  CONTEXT:                                                                                                       │
│  ---                                                                                                            │
│  Source: https://github.com/langflow-ai/langflow/releases\nDate: 2025-07-10\n\n## 1.5.0.post1\n\n<!-- Release   │
│  notes generated using configuration in .github/release.yml at 1.5.0.post1 -->                                  │
│                                                                                                                 │
│  ## What's Changed                                                                                              │
│  ### ✨ New Features                                                                                            │
│  * feat: Add dynamic theming support to WatsonxAI icon by @Cristhianzl in                                       │
│  https://github.com/langflow-ai/langflow/pull/8935                                                              │
│  * feat: jigsawstack bundle integration by @Khurdhula-Harshavardhan in                                          │
│  https://github.com/langflow-ai/langflow/pull/8832                                                              │
│  * feat: enhance DataFrame Operations component with contains filter and modern UI by @rodrigosnader in         │
│  https://github.com/langflow-ai/langflow/pull/8838                                                              │
│  * feat: add DataFrame output to Structured Output component by @rodrigosnader in                               │
│  https://github.com/langflow-ai/langflow/pull/8842                                                              │
│  ### 🐛 Bug Fixes                                                                                               │
│  * fix: Improve modal layout responsiveness and overflow handling by @Cristhianzl in                            │
│  https://github.com/langflow-ai/langflow/pull/8936                                                              │
│  * fix: Improve flow export error handling and validation by @Cristhianzl in                                    │
│  https://github.com/langflow-ai/langflow/pull/8943                                                              │
│  * fix: make deletion of single file commit to DB, create tests for file deletion by @lucaseduoli in            │
│  https://github.com/langflow-ai/langflow/pull/8953                                                              │
│  * fix: update API Access codes to include only authenticated code by @lucaseduoli in                           │
│  https://github.com/langflow-ai/langflow/pull/8957                                                              │
│  * fix: Ensure flow_id is not None before logging vertex build details by @ogabrielluiz in                      │
│  https://github.com/langflow-ai/langflow/pull/8954\n\n---\n\nSource:                                            │
│  https://github.com/langflow-ai/langflow/releases\nDate: 2025-07-10\n\n* fix: Ensure flow_id is not None        │
│  before logging vertex build details by @ogabrielluiz in 

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Principal Technology Analyst                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### Release Notes for Langflow (July 2025)                                                                     │
│                                                                                                                 │
│  ---                                                                                                            │
│                                                                                                                 │
│  ### **Version 1.5.0.post1 (July 10, 2025)**                                                                    │
│                                                                                                                 │
│  #### ✨ **New Features**                                                                                       │
│  - **Dynamic theming support** added to WatsonxAI icon                                                          │
│  ([#8935](https://github.com/langflow-ai/langflow/pull/8935))                                                   │
│  - **Jigsawstack bundle integration** ([#8832](https://github.com/langflow-ai/langflow/pull/8832))              │
│  - Enhanced **DataFrame Operations component** with contains filter and modern UI                               │
│  ([#8838](https://github.com/langflow-ai/langflow/pull/8838))                                                   │
│  - Added **DataFrame output** to Structured Output component                                                    │
│  ([#8842](https://github.com/langflow-ai/langflow/pull/8842))                                                   │
│                                                                                                                 │
│  #### 🐛 **Bug Fixes**                                                                                          │
│  - Improved **modal layout responsiveness** and overflow handling                                               │
│  ([#8936](https://github.com/langflow-ai/langflow/pull/8936))                                                   │
│  - Enhanced **flow export error handling** and validation                                                       │
│  ([#8943](https://github.com/langflow-ai/langflow/pull/8943))                                                   │
│  - Fixed **deletion of single file** commit to DB and created tests for file deletion                           │
│  ([#8953](https://github.com/langflow-ai/langflow/pull/8953))                                                   │
│  - Updated **API Access codes** to include only authenticated code                                              │
│  ([#8957](https://github.com/langflow-ai/langflow/pull/8957))                                                   │
│  - Ensured **flow_id is not None before logging vertex build details**                                          │
│  ([#8954](https://github.com/langflow-ai/langflow/pull/8954))                                                   │
│  - Fixed **max height of template getting started cards**                                                       │
│  ([#8964](https://github.com/langflow-ai/langflow/pull/8964))                                                   │
│  - Cleaned edges of same type but different name when switching outputs, updated color when deleting edges      │
│  ([#8959](https://github.com/langflow-ai/langflow/pull/89

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Communications Dispatch Officer                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Successfully sent personalized HTML emails to 3/3 recipients.                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 5436099e-77e4-4730-85ba-29794020261a                                                                     │
│  Agent: Communications Dispatch Officer                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 229378c8-caa1-4d63-b748-9f4cc417f3d3                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Successfully sent personalized HTML emails to 3/3 recipients.                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Crew execution for Langflow finished successfully!
Final Result: Successfully sent personalized HTML emails to 3/3 recipients.

Starting process for: SimpliDots
No monthly updates found for SimpliDots. Skipping.

Starting process for: Anthropic
No monthly updates found for Anthropic. Skipping.
